### Importing libraries

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

### Setting up Dataframe from data

In [2]:
postcode = []
borough = []
Neighbourhood = []

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", attrs={"class": "wikitable"})
rows = table.tbody.find_all("td")

for i in range(0,len(rows),3):
    postcode.append(rows[i].text.replace('\n', '').strip())
    borough.append(rows[i+1].text.replace('\n', '').strip())
    Neighbourhood.append(rows[i+2].text.replace('\n', '').strip())
    
temp_df = pd.DataFrame(postcode,columns=["Postcode"])
temp_df['borough'] = borough
temp_df['neighbourhood'] = Neighbourhood

### Exporting data to CSV for later use

In [3]:
temp_df.to_csv("./torrano.csv",index=False)

### Importing Data from CSV and dropping rows with borough = Not Assigned and moodifying CSV again

In [4]:
torrano_df = pd.read_csv("./torrano.csv")
torrano_df = torrano_df[torrano_df['borough'] != 'Not assigned'].reset_index(drop=True)
torrano_df.to_csv("./torrano.csv",index=False)

In [5]:
torrano_df.head()
torrano_df = torrano_df[['Postcode','borough','neighbourhood']].groupby(by=['Postcode','borough']).agg({'neighbourhood': ','.join})

In [6]:
torrano_df = torrano_df.reset_index()
torrano_df.head()

,Postcode,borough,neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Checking if neighbourhood column has any 'not assigned' value

In [7]:
torrano_df[torrano_df['neighbourhood'] == 'Not assigned']

,Postcode,borough,neighbourhood


Since we do not have any data, we can print shape

In [8]:
torrano_df.shape

(103, 3)

In [9]:
torrano_df.to_csv("./torrano.csv",index=False)